# CIFAR-10 Deep Convolutional GAN

In [ ]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image

In [ ]:
# Loading CIFAR 10 dataset

image_height = 32
image_width = 32
color_channels = 3

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

cifar_path = './cifar-10-python/'

train_data = np.array([])
train_labels = np.array([])

# Load all the data batches.
for i in range(1,6):
    data_batch = unpickle(cifar_path + 'data_batch_' + str(i))
    train_data = np.append(train_data, data_batch[b'data'])
    train_labels = np.append(train_labels, data_batch[b'labels'])
    
# TODO: Process Cifar data
 
def process_data(data):
    float_data = np.array(data, dtype=float) / 255.0
     
    reshaped_data = np.reshape(float_data, (-1, color_channels, image_height, image_width))
    transposed_data = np.transpose(reshaped_data, [0, 2, 3, 1])
     
    return transposed_data
 
train_data = process_data(train_data)

# TODO: Find labels of only the class we want

train_class = 8

class_train_data = []

indices = [i for i, x in enumerate(train_labels) if x == train_class]

for i in indices:
    class_train_data.append(train_data[i])
    
class_train_data = np.array(class_train_data)
print(class_train_data.shape)

In [ ]:
# Saving image samples
def save_sample(images_array, filename, shape):
    
    img_width = images_array.shape[1]
    img_height = images_array.shape[2]
    
    final_width = img_width * shape[0]
    final_height = img_width * shape[1]
    
    color_channels = images_array.shape[3]
    
    final_arr = np.zeros((final_width, final_height, color_channels))
    
    for i in range(len(images_array)):
        x = int(i % shape[0]) * img_width
        y = int(i / shape[0]) * img_height
        
        final_arr[x:x + img_width, y:y + img_height] = images_array[i].reshape(img_height, img_width, color_channels)
        
    final_img = Image.fromarray((final_arr * 255).astype(np.uint8), mode="RGB")
    final_img.save(filename)

In [ ]:
# Generator Network

def create_generator(z, reuse=False):
    with tf.variable_scope("generator", reuse=reuse) as scope:
        dense = tf.layers.dense(z, 4096, kernel_initializer=initializer, bias_initializer=initializer)
        norm = tf.contrib.layers.batch_norm(dense)
        a = tf.nn.relu(norm)
        conv_input = tf.reshape(a, [-1, 4, 4, 256])
        
        conv = tf.layers.conv2d_transpose(conv_input, 128, [5, 5], strides=2, padding="same",
                                          bias_initializer=initializer, kernel_initializer=initializer)
        norm = tf.contrib.layers.batch_norm(conv)
        a = tf.nn.relu(norm)
        
        conv = tf.layers.conv2d_transpose(a, 64, [5, 5], strides=2, padding="same",
                                          bias_initializer=initializer, kernel_initializer=initializer)
        norm = tf.contrib.layers.batch_norm(conv)
        a = tf.nn.relu(norm)
        
        conv = tf.layers.conv2d_transpose(a, color_channels, [5, 5], strides=2, padding="same",
                                          bias_initializer=initializer, kernel_initializer=initializer)
        norm = tf.contrib.layers.batch_norm(conv)
        a = tf.nn.relu(norm)
        
        conv = tf.layers.conv2d_transpose(a, color_channels, [7, 7], padding="same",
                                          bias_initializer=initializer, kernel_initializer=initializer)
        output = tf.sigmoid(conv)
        
    return output

In [ ]:
# Discriminator Network

def create_discriminator(image, reuse=False):
    with tf.variable_scope("discriminator", reuse=reuse) as scope:
        conv = tf.layers.conv2d(image, 64, [5, 5], padding="same",
                                bias_initializer=initializer, kernel_initializer=initializer)
        norm = tf.contrib.layers.batch_norm(conv)
        a = tf.nn.leaky_relu(norm)
        pool = tf.layers.max_pooling2d(a, [2, 2], [2, 2], "same")
        
        conv = tf.layers.conv2d(pool, 128, [5, 5], padding="same",
                                bias_initializer=initializer, kernel_initializer=initializer)
        norm = tf.contrib.layers.batch_norm(conv)
        a = tf.nn.leaky_relu(norm)
        pool = tf.layers.max_pooling2d(a, [2, 2], [2, 2], "same")
        
        conv = tf.layers.conv2d(pool, 128, [5, 5], padding="same",
                                bias_initializer=initializer, kernel_initializer=initializer)
        norm = tf.contrib.layers.batch_norm(conv)
        a = tf.nn.leaky_relu(norm)
        pool = tf.layers.max_pooling2d(a, [2, 2], [2, 2], "same")
        flatten = tf.layers.flatten(pool)
        
        dense = tf.layers.dense(flatten, 1, kernel_initializer=initializer, bias_initializer=initializer)
        output = tf.sigmoid(dense)
        
    return output

In [ ]:
# TODO: Build the model
tf.reset_default_graph()

z_dim = 100

initializer = tf.truncated_normal_initializer(stddev=0.02)

x_images = tf.placeholder(dtype=tf.float32, shape=[None, image_height, image_width, color_channels])
z_noise = tf.placeholder(dtype=tf.float32, shape=[None, z_dim])

Gz = create_generator(z_noise)
Dx = create_discriminator(x_images)
DGz = create_discriminator(Gz, reuse=True)

g_loss = -tf.reduce_mean(tf.log(DGz))
d_loss = -tf.reduce_mean(tf.log(Dx) + tf.log(1 - DGz))

g_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="generator")
d_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="discriminator")

g_train = tf.train.AdamOptimizer(learning_rate=0.0002, beta1=0.5).minimize(g_loss, var_list=g_vars)
d_train = tf.train.AdamOptimizer(learning_rate=0.0002, beta1=0.5).minimize(d_loss, var_list=d_vars)

In [ ]:
# Training

load_checkpoint = False
path = "GAN checkpoints/"
saver = tf.train.Saver(max_to_keep=2)

model_name = "cifar-gan"

batch_size = 200
epochs = 15000
display_step = 10

init = tf.global_variables_initializer()
sess = tf.Session()
if load_checkpoint:
    checkpoint = tf.train.get_checkpoint_state(path)
    saver.restore(sess, checkpoint.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

current_batch_index = 0

test_z = np.random.uniform(-1.0, 1.0, size=[25, z_dim]).astype(np.float32)
test_gen = create_generator(z_noise, reuse=True)

for epoch in range(epochs):
    batch_xs = class_train_data[current_batch_index:current_batch_index + batch_size]
    
    if current_batch_index + batch_size >= len(class_train_data):
        current_batch_index = 0
    else:
        current_batch_index += batch_size
        
    zs = np.random.uniform(-1.0, 1.0, size=[batch_size, z_dim]).astype(np.float32)
        
    if epoch % display_step == 0:
        a = np.array(sess.run(test_gen, feed_dict={z_noise: test_z}))
        save_sample(a, "Generated/" + str(epoch) + ".bmp", [5, 5])
        saver.save(sess, path + model_name, epoch)
        
    sess.run(d_train, feed_dict={x_images: batch_xs, z_noise: zs})
    sess.run(g_train, feed_dict={z_noise: zs})
    sess.run(g_train, feed_dict={z_noise: zs})
    
    print("Epoch", 
          epoch, 
          "Generator Loss", 
          sess.run(g_loss, 
                   feed_dict={z_noise: zs}), 
          "Discriminator Loss", 
          sess.run(d_loss, 
                   feed_dict={x_images: batch_xs, z_noise: zs}))
    
saver.save(sess, path + model_name, epoch)